Define TimeContrastiveModel

In [1]:
import torch
from torch import nn
import numpy as np

class Convblock(nn.Module):
    def __init__(self, in_channels,hidden_channels,out_channels):
        super(Convblock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, hidden_channels,
                               kernel_size=5)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(hidden_channels, out_channels,
                               kernel_size=5)
        self.relu2 = nn.ReLU()

    def forward(self, x):
        # reshape
        x = x.view(-1,288,42,77) # x.view(-1,288,35,35) for 299 x 299 else 360 x 640
        x = self.conv1(x)
        # Activation function
        x = self.relu1(x)
        # Second convolutional layer
        x = self.conv2(x)
        # Activation function
        x = self.relu2(x)
        return x

# spatial Softmax and last linear layer
class SpatialSoftmax(torch.nn.Module):
    def __init__(self, height, width, channel, temperature=None):
        super(SpatialSoftmax, self).__init__()
        self.height = height
        self.width = width
        self.channel = channel

        if temperature:  
          self.temperature = torch.ones(1)*temperature   
        else:   
          self.temperature = nn.Parameter(torch.ones(1))  

        pos_x, pos_y = np.meshgrid(
                np.linspace(-1., 1., self.height),
                np.linspace(-1., 1., self.width)
                )
        pos_x = torch.from_numpy(pos_x.reshape(self.height*self.width)).float()
        pos_y = torch.from_numpy(pos_y.reshape(self.height*self.width)).float()
        self.register_buffer('pos_x', pos_x)
        self.register_buffer('pos_y', pos_y)
        self.linear =nn.Linear(32, 32)

    def forward(self, feature):
        feature = feature.view(-1,self.height*self.width)

        softmax_attention = nn.functional.softmax(feature/self.temperature, dim=-1)
        expected_x = torch.sum(self.pos_x*softmax_attention, dim=1, keepdim=True)
        expected_y = torch.sum(self.pos_y*softmax_attention, dim=1, keepdim=True)
        expected_xy = torch.cat([expected_x, expected_y], 1)
        feature_keypoints = expected_xy.view(-1, self.channel*2)
        feature_keypoints = self.linear(feature_keypoints)

        return feature_keypoints

# TCNmodel
class TCNmodel(nn.Module):
  # input tensor: N x 3 x H(>=299) x W(>=299) 
  # output tensor: N x 32


    def __init__(self):
        super(TCNmodel, self).__init__()
        self.model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)
        self.conv2 = Convblock(288,32,16)
        self.spmax = SpatialSoftmax(34,69,16,temperature=None) #SpatialSoftmax(27,27,16,temperature=None) for 299 x 299 else for 360 x 640

    def forward(self, x):
        x = self.model.Conv2d_1a_3x3(x)
        x = self.model.Conv2d_2a_3x3(x)
        x = self.model.Conv2d_2b_3x3(x)
        x = self.model.maxpool1(x)

        x = self.model.Conv2d_3b_1x1(x)
        x = self.model.Conv2d_4a_3x3(x)
        x = self.model.maxpool2(x)
        x = self.model.Mixed_5b(x)
        x = self.model.Mixed_5c(x)
        x = self.model.Mixed_5d(x)

        x = self.conv2(x)
        x = self.spmax(x)

        return x

In [28]:
# #testing model
# model = TCNmodel()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [1]:
# import urllib
# url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
# try: urllib.URLopener().retrieve(url, filename)
# except: urllib.request.urlretrieve(url, filename)

# # sample execution (requires torchvision)
# from PIL import Image
# from torchvision import transforms
# input_image = Image.open(filename)
# preprocess = transforms.Compose([
#     transforms.Resize([360,640]),
#     #transforms.CenterCrop(299),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])
# print(input_image.size)

# # input_tensor = preprocess(input_image)
# # input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
# # print(input_batch.shape)
# # input_batch=torch.cat(10*[input_batch])
# # print(input_batch.shape)
# # #input_batch = torch.cat((input_batch,input_batch),dim=0)
# # # move the input and model to GPU for speed if available
# # if torch.cuda.is_available():
# #     input_batch = input_batch.to('cuda')
# #     model.to('cuda')

# # with torch.no_grad():
# #   output = model(input_batch)
# # # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
# # print(output.shape)

(1546, 1213)
